In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split
from skimage import *
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2grey

In [10]:
def load_image_files(container_path, dimension=(32,32)):
    """
    Load image files with categories as subfolder names 
    which performs like scikit-learn sample dataset
    
    Parameters
    ----------
    container_path : string or unicode
        Path to the main folder holding one subfolder per category
    dimension : tuple
        size to which image are adjusted to
        
    Returns
    -------
    Bunch
    """
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []
    for i, direc in enumerate(folders):
        
        for file in direc.iterdir():
            img = io.imread(file,plugin='matplotlib')
            #color_features = img.flatten()
            grey_image = rgb2grey(img)
            img_resized = resize(grey_image, dimension, anti_aliasing=True, mode='reflect')
            #plt.imshow(img_resized)
            flat_data.append(img_resized.flatten()) 
            images.append(img_resized)
            target.append(i)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

In [11]:
train_dataset = load_image_files("/Users/macbookpro/Documents/Masters/Deep Learning/dataset/training_set/")

In [4]:
validation_dataset = load_image_files("/Users/macbookpro/Documents/Masters/Deep Learning/dataset/validation_set/")

In [12]:
G = 0.01
CC = 2
svm = SVC(kernel='rbf', probability=True, gamma=G,C = CC,random_state = 1)
svm.fit(train_dataset.data, train_dataset.target)

SVC(C=2, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=True, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

In [13]:
y_pred = svm.predict(validation_dataset.data)
accuracy = accuracy_score(validation_dataset.target, y_pred)
print('Model accuracy is: ', accuracy)

Model accuracy is:  0.4153846153846154


In [14]:
from sklearn.metrics import confusion_matrix
cfm = confusion_matrix(validation_dataset.target, y_pred)
print(cfm)

[[ 70  24  14  15  26  30  15  11]
 [ 17  72  13   9  13  32  25  26]
 [  8  14  50   2  20  32  33  43]
 [ 19  13  16 102  12  17  13   5]
 [ 26   2   3   3 128  19   8  11]
 [ 18  10  21  10  31  72  16  25]
 [ 11  26  23  14  16  19  78  18]
 [  4  14  21   3   7  20  34 103]]


In [16]:
#cfm_n = confusion_matrix(np.argmax(model_n.predict(X_test), axis=1), np.argmax(y_test, axis=1))
cfm_n = np.flipud(cfm)
cfm_n = cfm_n.astype('float') / cfm_n.sum(axis=1)[:, np.newaxis]
cfm_n = np.round(cfm_n,2)

In [19]:
import plotly.figure_factory as ff

z = cfm_n
x = ['animals','art','automobiles','fashion','food','houses','technology','travel']
y = ['travel','technology','houses','food','fashion','automobiles','art','animals']

fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z, colorscale='deep')
fig.update_layout(width=700, height=700, xaxis_showgrid=False, yaxis_showgrid=False, template='none')
fig.update_layout(title={
        #'text': "Results using 1D CNN",
        'y':0.95,
        'x':0.47,
        'xanchor': 'center',
        'yanchor': 'top'},
        xaxis_title='predicted label',
        yaxis_title='true label')
fig['layout']['xaxis']['side'] = 'bottom'
fig.show()

In [20]:
fig.show()